# Explore BL Sessions Data

Here we introduce some useful DataJoint tools to explore the bl19 pipeline

First thing first, import datajoint

In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()


Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [2]:
import datajoint as dj
import utility.blob_transformation as bt
import numpy as np
import pylab as plt
import matplotlib.patches as mpatches
from matplotlib import cm

import pandas as pd

In [3]:
dj.blob.use_32bit_dims = True

In [4]:
dj.conn(host='bdata00.pni.princeton.edu',reset=True)

[2022-12-09 10:27:44,802][INFO]: Connecting alvaros@bdata00.pni.princeton.edu:3306
[2022-12-09 10:27:45,952][INFO]: Connected alvaros@bdata00.pni.princeton.edu:3306


DataJoint connection (connected) alvaros@bdata00.pni.princeton.edu:3306

In [5]:
bdata = dj.create_virtual_module('bdata', 'bdata')

In [6]:
session_key = {'sessid': 898802}
df_events = pd.DataFrame((bdata.BehaviorEvent & session_key).fetch(as_dict=True))
df_events

""


## Get handle to DBs

lab         =  researchers & rig DB <br />
subject     =  rats DB <br />
acquisition =  sessions DB

In [7]:
# session_data = (bdata.Sessions & session_key).fetch('protocol_data', as_dict=True)
parsed_events = (bdata.ParsedEvents & session_key).fetch(as_dict=True)

In [9]:
protocol_data_python = bt.transform_blob(session_data[0]['protocol_data'])
df_protocol_data = bt.blob_dict_to_df(protocol_data_python)

NameError: name 'session_data' is not defined

In [10]:
peh = bt.transform_blob(parsed_events[0]['peh'])
df_peh = bt.blob_peh_to_df(peh, append_original_columnname=True)
df_peh

,pokes__C,pokes__L,pokes__R,pokes__starting_state,pokes__ending_state,waves__cp_fixation_wave,waves__sa_sound_wave,waves__sb_sound_wave,waves__go_cue_wave,waves__go_corr_light_wave,...,states__retry_hit_state,states__error_state,states__violation_state,states__violation_penalty_state,states__general_final_state,states__hit_final_state,states__error_final_state,states__violation_final_state,states__starting_state,states__ending_state
0,"[100.373604, 100.869204]",[],"[100.999004, 102.76760399999999]","{'C': 'out', 'L': [], 'R': 'out'}","{'C': 'out', 'L': [], 'R': 'out'}",[],"[100.573604, 100.919304]",[],[],[],...,[],[],"[100.919204, 100.920204]","[100.920204, 101.920104]","[101.920104, 102.920104]",[],[],[],state_0,state_0
1,"[110.139004, 110.639304]",[],"[[109.652204, 109.93240399999999], [110.774903...","{'C': 'out', 'L': [], 'R': 'out'}","{'C': 'out', 'L': [], 'R': 'in'}","[110.53900399999999, 112.53900399999999]","[110.33900399999999, 110.739004]","[111.139004, 111.53900399999999]","[111.739004, 113.739004]","[111.739204, 111.73980399999999]",...,[],[],[],[],"[116.829404, 117.829404]",[],[],[],state_0,state_0
2,"[[121.417304, 121.867704], [125.842404, nan]]","[nan, 121.105604]","[[122.00140400000001, 124.337904], [124.818704...","{'C': 'out', 'L': 'in', 'R': 'in'}","{'C': 'in', 'L': 'out', 'R': 'out'}","[121.81730400000001, 123.81730400000001]","[121.617304, 122.017304]","[122.417304, 122.81730400000001]","[123.017304, 125.017304]","[123.017504, 123.018104]",...,[],"[123.018004, 125.018004]",[],[],"[125.018004, 126.018004]",[],[],[],state_0,state_0
3,"[[130.83540399999998, 131.25480399999998], [13...",[],"[[131.410104, 133.413004], [134.589804, nan]]","{'C': 'in', 'L': 'out', 'R': 'out'}","{'C': 'out', 'L': 'out', 'R': 'in'}","[131.235404, 133.235404]","[131.035404, 131.43540399999998]","[131.83540399999998, 132.235404]","[132.43540399999998, 134.43540399999998]","[132.43560399999998, 132.43620399999998]",...,[],"[132.436104, 134.436104]",[],[],"[134.436104, 135.436104]",[],[],[],state_0,state_0
4,"[[139.947204, 140.172504], [140.179204, 140.20...","[140.415104, nan]",[],"{'C': 'out', 'L': 'out', 'R': 'in'}","{'C': 'out', 'L': 'in', 'R': 'in'}",[],"[140.14720400000002, 140.259904]",[],[],[],...,[],[],"[140.259804, 140.260804]","[140.260804, 141.260704]","[141.260704, 142.260704]",[],[],[],state_0,state_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,"[3526.4605030000002, 3526.9450030000003]",[],"[3527.114203, 3528.957503]","{'C': 'out', 'L': 'out', 'R': 'out'}","{'C': 'out', 'L': 'out', 'R': 'out'}",[],"[3526.660503, 3526.995103]",[],[],[],...,[],[],"[3526.995003, 3526.996003]","[3526.996003, 3527.995903]","[3527.995903, 3528.995903]",[],[],[],state_0,state_0
274,"[3535.093303, 3535.4650030000003]",[],"[3535.612803, nan]","{'C': 'out', 'L': 'out', 'R': 'out'}","{'C': 'out', 'L': 'out', 'R': 'in'}",[],"[3535.293303, 3535.515103]",[],[],[],...,[],[],"[3535.515003, 3535.516003]","[3535.516003, 3536.515903]","[3536.515903, 3537.515903]",[],[],[],state_0,state_0
275,"[3547.868903, 3548.265203]","[3548.5342029999997, nan]",[],"{'C': 'out', 'L': 'out', 'R': 'in'}","{'C': 'out', 'L': 'in', 'R': 'in'}",[],"[3548.068903, 3548.315303]",[],[],[],...,[],[],"[3548.315203, 3548.316203]","[3548.316203, 3549.316103]","[3549.316103, 3550.316103]",[],[],[],state_0,state_0
276,"[3555.513903, 3555.895803]","[3556.109903, nan]",[],"{'C': 'out', 'L': 'in', 'R': 'in'}","{'C': 'out', 'L': 'in', 'R': 'in'}",[],"[3555.7139030000003, 3555.9459030000003]",[],[],[],...,[],[],"[3555.945803, 3555.9468030000003]","[3555.9468030000003, 3556.946703]","[3556.946703, 3557.946703]",[],[],[],state_0,state_0


In [11]:
df_peh.loc[:, 'states__wait_for_go'].tail(26)

252                                   []
253                                   []
254    [3335.900004, 3337.1000040000004]
255                                   []
256           [3354.254904, 3355.454904]
257           [3363.490804, 3364.690804]
258    [3372.901304, 3374.1013040000003]
259           [3381.622304, 3382.822304]
260                                   []
261           [3399.430704, 3400.630704]
262                                   []
263           [3418.214904, 3419.414904]
264    [3429.500404, 3430.7004039999997]
265                                   []
266           [3449.852704, 3451.052704]
267    [3463.861504, 3465.0615040000002]
268                                  NaN
269                                  NaN
270                                  NaN
271                                  NaN
272                                  NaN
273                                  NaN
274                                  NaN
275                                  NaN
276             

In [12]:
df_event = bt.peh_trial_df_to_event_df(df_peh, session_key['sessid'])
df_event

                                              pokes__C  is_numpy
0                             [100.373604, 100.869204]      True
1                             [110.139004, 110.639304]      True
2        [[121.417304, 121.867704], [125.842404, nan]]      True
3    [[130.83540399999998, 131.25480399999998], [13...      True
4    [[139.947204, 140.172504], [140.179204, 140.20...      True
..                                                 ...       ...
273           [3526.4605030000002, 3526.9450030000003]      True
274                  [3535.093303, 3535.4650030000003]      True
275                         [3547.868903, 3548.265203]      True
276                         [3555.513903, 3555.895803]      True
277                         [3563.135203, 3563.508003]      True

[278 rows x 2 columns]
pokes__C
                                              pokes__C  is_numpy
0                             [100.373604, 100.869204]      True
1                             [110.139004, 110.639304]   

AttributeError: 'float' object has no attribute 'size'